# Processing LAPD calls for service: 2010-2020

### Load python tools

In [182]:
import json
import pandas as pd
import matplotlib
import numpy as np
import altair as alt
from altair import datum
import altair_latimes as lat
pd.options.display.max_columns = 50
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', None)
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
alt.renderers.enable('default')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Download latest data from all sources

In [2]:
# %run 'download_calls.ipynb'

## Los Angeles Police

In [96]:
df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/la/los_angeles.csv', low_memory=False)

In [97]:
df.head()

,incident_number,reporting_district,area_occurred,dispatch_date,dispatch_time,call_type_code,call_type_description,date,time
0,PD20012600002716,Rampart,0245,01/26/2020,14:37:04,211S,SUSP,2020-01-26,14:37:04
1,PD20011400005678,Outside,NaN,01/14/2020,23:26:20,006,CODE 6,2020-01-14,23:26:20
2,PD20011100005314,Outside,NaN,01/11/2020,23:32:40,006,CODE 6,2020-01-11,23:32:40
3,PD20010100005220,Southwest,0328,01/01/2020,22:34:29,245SN,SUSP NOW,2020-01-01,22:34:29
4,PD20010500000660,Hollywood,0636,01/05/2020,04:04:42,907PA2,POSS AMB O/D,2020-01-05,04:04:42


In [98]:
df['date'] = pd.to_datetime(df['dispatch_date'], format='%m/%d/%Y')
df['monthday'] = df['date'].dt.strftime('%m/%d')
df['year'] = df['date'].dt.strftime('%Y')

In [99]:
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['date'] = df['date'].dt.date

In [100]:
df = df.drop(['dispatch_time', 'dispatch_date'], axis=1)

In [101]:
los_angeles = df.copy()

---

In [114]:
los_angeles.head()

,incident_number,reporting_district,area_occurred,call_type_code,call_type_description,date,time,monthday,year,month,day
0,PD20012600002716,Rampart,0245,211S,SUSP,2020-01-26,14:37:04,01/26,2020,1,26
1,PD20011400005678,Outside,NaN,006,CODE 6,2020-01-14,23:26:20,01/14,2020,1,14
2,PD20011100005314,Outside,NaN,006,CODE 6,2020-01-11,23:32:40,01/11,2020,1,11
3,PD20010100005220,Southwest,0328,245SN,SUSP NOW,2020-01-01,22:34:29,01/01,2020,1,1
4,PD20010500000660,Hollywood,0636,907PA2,POSS AMB O/D,2020-01-05,04:04:42,01/05,2020,1,5


In [241]:
this_year = los_angeles[los_angeles['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls').drop([59], axis=0)

### What changed? 

In [423]:
this_year_before = this_year[(this_year['monthday'] < '05/30')]
protest_period = pd.DataFrame(this_year[(this_year['monthday'] > '05/27') & (this_year['monthday'] < '06/10')])

In [424]:
protest_period.calls.mean()

2975.6153846153848

In [425]:
this_year_before.calls.max()

6724

In [526]:
this_year.calls.mean()

4943.526315789473

In [427]:
protest_period['mean_before'] = this_year_before.calls.max()

In [428]:
protest_period

,monthday,calls,mean_before
148,05/28,5028,6724
149,05/29,4128,6724
150,05/30,2435,6724
151,05/31,2611,6724
152,06/01,2702,6724
153,06/02,2618,6724
154,06/03,2463,6724
155,06/04,2485,6724
156,06/05,2761,6724
157,06/06,2594,6724


In [537]:
chart = alt.Chart((this_year))\
    .mark_area(size=3,opacity=.6, color='#B32F2E', interpolate='monotone')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 6000)))
).properties(width=630, height=400,
     title='All service calls to LAPD: 2020'
)

rule = alt.Chart(this_year).mark_rule(color='#666', size=1).encode(
    y='mean(calls):Q',
)

text = rule.mark_text(
    color="#000000", font='Benton Gothic', fontSize=12,
    align='center',
    baseline='middle',
    dx=112,
    dy=12,
    fontWeight='bold',
).encode(
    text=alt.Text('label:N', format="")).transform_calculate(label='"Average: 4,900"')

text_protest = rule.mark_text(
    color="#000000", font='Benton Gothic', fontSize=12,
    align='center',
    baseline='middle',
    dx=252,
    dy=-130,
    fontWeight='bold',
).encode(
    text=alt.Text('label:N', format="")).transform_calculate(label='"Protests"')

protest_annotation = alt.Chart(protest_period).mark_area(color='#999', opacity=.3).encode(
    y='mean_before',
    x='monthday:T'
)

(protest_annotation + chart + text + text_protest + rule).configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.LayerChart(...)

### What was the percentage change

In [430]:
(protest_period.calls.mean() - this_year_before.calls.mean())/this_year_before.calls.mean()

-0.4233551024581533

### Which types of calls dropped the most during the protest period?

### What are these 'Code 6' calls?

In [146]:
no_code6 = los_angeles[los_angeles['reporting_district'] != 'Outside']

In [158]:
this_year_no_code6 = no_code6[no_code6['year'] == '2020'].groupby(['monthday', 'reporting_district']).agg('size').reset_index(name='calls')

In [214]:
this_year_fireworks = los_angeles[(los_angeles['year'] == '2020') & (los_angeles['call_type_description'] == 'FIREWORKS')].groupby(['monthday', 'reporting_district']).agg('size').reset_index(name='calls')

In [262]:
chart = alt.Chart((this_year_fireworks))\
    .mark_bar(size=1,opacity=1, color='#B32F2E')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=3, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 30))),
    facet=alt.Facet('reporting_district:O', columns=7, title='LAPD reporting district')
).properties(width=100, height=100,
     title='All fireworks calls to LAPD: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [232]:
chart = alt.Chart((this_year_no_code6))\
    .mark_bar(size=1,opacity=1, color='#B32F2E')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 300))),
    facet=alt.Facet('reporting_district:O', columns=7, title='LAPD reporting district')
).properties(width=150, height=100,
     title='All fireworks calls to LAPD: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

### Which cases are related to fireworks?

In [200]:
fireworks = pd.DataFrame(df[df['call_type_code'] == '507F'])

In [201]:
daily = fireworks[fireworks['year'] != '2020'].groupby(['monthday']).agg('size').reset_index(name='average')

In [202]:
daily['average'] = daily['average']/10

In [203]:
thisyear = fireworks[fireworks['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='average')

In [204]:
fireworks_trend = daily.merge(thisyear, on=['monthday'], how='left')

In [205]:
fireworks_trend.rename(columns={'average_x':'Average 2010-2019', 'average_y':'ThisYear'}, inplace=True)

In [206]:
fireworks_trend.ThisYear = fireworks_trend.ThisYear.ffill()

In [207]:
fireworks_chart = pd.DataFrame(fireworks_trend[fireworks_trend['monthday'] < '06/20'])

In [208]:
fireworks_melt = pd.melt(fireworks_chart, id_vars=['monthday'], value_vars=['Average 2010-2019', 'ThisYear'],
        var_name='time', value_name='calls')

In [245]:
chart = alt.Chart((fireworks_melt))\
    .mark_bar(size=4,opacity=0.4)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 200))),
    color=alt.Color("time:N", title=' ')
).properties(width=630, height=400,
     title='Fireworks calls to LAPD: 2010-2019'
)

(chart).configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square', orient='top')

alt.Chart(...)

In [210]:
fireworks_chart['monthday'] = fireworks_chart['monthday'] + '/2020'

In [538]:
fireworks_chart.to_csv('output/fireworks_calls_datawrapper.csv', index=None)

In [540]:
this_year['monthday'] = this_year['monthday'] + '/2020'

In [541]:
this_year.to_csv('output/all_calls_datawrapper.csv', index=None)

---

## San Francisco

In [24]:
df_sf = pd.read_csv('/Users/mhustiles/data/data/LA/calls/san-francisco/san_francisco.csv', low_memory=False)

In [25]:
df_sf['date'] = pd.to_datetime(df_sf['date_time'], format='%Y-%m-%d %H:%M:%S')
df_sf['time'] = pd.to_datetime(df_sf['date_time'], format='%Y-%m-%d %H:%M:%S')

In [26]:
df_sf['monthday'] = df_sf['date'].dt.strftime('%m/%d')
df_sf['year'] = df_sf['date'].dt.strftime('%Y')

In [27]:
df_sf.head()

,crime_id,original_crime_type_name,report_date,call_date,offense_date,call_time,call_date_time,disposition,address,city,state,agency_id,address_type,common_location,date_time,date,time,monthday,year
0,193043877,Vandalism,10/31/2019,10/31/2019,10/31/2019,20:46,10/31/2019 08:46:00 PM,UTL,300 Block Of Toland St,San Francisco,CA,1,Premise Address,NaN,2019-10-31 08:46:00,2019-10-31 08:46:00,2019-10-31 08:46:00,10/31,2019
1,190020633,Trespasser,01/02/2019,01/02/2019,01/02/2019,07:58,01/02/2019 07:58:00 AM,GOA,300 Block Of Industrial St,San Francisco,CA,1,Premise Address,NaN,2019-01-02 07:58:00,2019-01-02 07:58:00,2019-01-02 07:58:00,01/02,2019
2,190214007,Passing Call,01/21/2019,01/21/2019,01/21/2019,23:07,01/21/2019 11:07:00 PM,HAN,Larkin St/golden Gate Av,San Francisco,CA,1,Intersection,NaN,2019-01-21 11:07:00,2019-01-21 11:07:00,2019-01-21 11:07:00,01/21,2019
3,190271719,22500e,01/27/2019,01/27/2019,01/27/2019,12:48,01/27/2019 12:48:00 PM,GOA,3400 Block Of Divisadero St,San Francisco,CA,1,Premise Address,NaN,2019-01-27 12:48:00,2019-01-27 12:48:00,2019-01-27 12:48:00,01/27,2019
4,190390109,Hot,02/08/2019,02/08/2019,02/08/2019,00:55,02/08/2019 12:55:00 AM,NOM,700 Block Of Corbett Av,San Francisco,CA,1,Premise Address,NaN,2019-02-08 12:55:00,2019-02-08 12:55:00,2019-02-08 12:55:00,02/08,2019


In [28]:
this_year_sf = df_sf[df_sf['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [29]:
chart = alt.Chart((this_year_sf))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 2500)))
).properties(width=630, height=400,
     title='All service calls to San Francisco Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [60]:
fireworks_melt.to_csv('output/fireworks_melt.csv')

In [31]:
this_year_sf.sort_values(by='monthday', ascending=False).head()

,monthday,calls
163,06/12,1531
162,06/11,1730
161,06/10,1759
160,06/09,1872
159,06/08,1737


---

## Sacramento

In [32]:
df_sac = pd.read_csv('/Users/mhustiles/data/data/LA/calls/sacramento/sacramento.csv', low_memory=False)

In [33]:
df_sac['date'] = pd.to_datetime(df_sac['date'], format='%Y-%m-%d')
df_sac['time'] = pd.to_datetime(df_sac['time'], format='%H:%M:%S')

In [34]:
df_sac['monthday'] = df_sac['date'].dt.strftime('%m/%d')
df_sac['year'] = df_sac['date'].dt.strftime('%Y')

In [35]:
this_year_sac = df_sac[df_sac['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [58]:
chart = alt.Chart((this_year_sac))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 1500)))
).properties(width=630, height=400,
     title='All service calls to Sacramento Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [37]:
this_year_sac.sort_values(by='monthday', ascending=False).head(5)

,monthday,calls
162,06/11,306
161,06/10,893
160,06/09,858
159,06/08,771
158,06/07,788


---

### Santa Monica

In [292]:
sm_df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/santa_monica/santa_monica.csv', low_memory=False)

In [293]:
sm_df['date'] = pd.to_datetime(sm_df['received_time'], format='%Y-%m-%d %H:%M:%S')

In [294]:
sm_df['monthday'] = sm_df['date'].dt.strftime('%m/%d')
sm_df['year'] = sm_df['date'].dt.strftime('%Y')

In [295]:
this_year_sm = sm_df[sm_df['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [296]:
chart = alt.Chart((this_year_sm))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 700)))
).properties(width=630, height=400,
     title='All service calls to Santa Monica Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

---

### San Diego

In [297]:
sd_df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/san-diego/san_diego.csv', low_memory=False)

In [298]:
sd_df.head()

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority,date,time
0,E20010000001,2020-01-01 00:00:09,4,400,NaN,06TH,AVE,NaN,NaN,NaN,11-8,A,523,0,2020-01-01,00:00:09
1,E20010000002,2020-01-01 00:00:20,4,5000,NaN,UNIVERSITY,AVE,NaN,NaN,NaN,FD,K,826,2,2020-01-01,00:00:20
2,E20010000003,2020-01-01 00:00:21,4,800,NaN,SAWTELLE,AVE,NaN,NaN,NaN,AU1,W,434,1,2020-01-01,00:00:21
3,E20010000004,2020-01-01 00:00:32,4,5000,NaN,UNIVERSITY,AVE,NaN,NaN,NaN,FD,K,826,2,2020-01-01,00:00:32
4,E20010000005,2020-01-01 00:00:42,4,5200,NaN,CLAIREMONT MESA,BLV,NaN,NaN,NaN,415V,K,111,1,2020-01-01,00:00:42


In [299]:
sd_df['date'] = pd.to_datetime(sd_df['date_time'], format='%Y-%m-%d %H:%M:%S')

In [300]:
sd_df['monthday'] = sd_df['date'].dt.strftime('%m/%d')
sd_df['year'] = sd_df['date'].dt.strftime('%Y')

In [301]:
this_year_sd = sd_df[sd_df['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [302]:
this_year_sd.loc[this_year_sd['monthday'] == '05/03', 'calls'] = 1557

In [303]:
chart = alt.Chart((this_year_sd.query('monthday < "06/16"')))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 2000)))
).properties(width=630, height=400,
     title='All service calls to San Diego Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

---

### San Jose

In [51]:
sj_df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/san-jose/san_jose.csv', low_memory=False)

In [52]:
sj_df['date'] = pd.to_datetime(sj_df['date'], format='%Y-%m-%d %H:%M:%S')

In [53]:
sj_df['monthday'] = sj_df['date'].dt.strftime('%m/%d')
sj_df['year'] = sj_df['date'].dt.strftime('%Y')

In [54]:
this_year_sj = sj_df[sj_df['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [55]:
chart = alt.Chart((this_year_sj))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 1000)))
).properties(width=630, height=400,
     title='All service calls to San Jose Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [56]:
sj_df.head()

,cdts,eid,start_date,call_number,priority,report_date,offense_date,offense_time,calltype_code,call_type,final_dispo_code,final_dispo,common_place_name,address,city,state,date,time,monthday,year
0,20200101002621PS,7981569,1/1/2020 12:00:00 AM,P200010004,2,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,00:03:08,1057,FIREARMS DISCHARGED,N,No report required; dispatch record only,NaN,[900]-[1000] FURLONG DR,San Jose,CA,2020-01-01,00:03:08,01/01,2020
1,20200101021606PS,7981674,1/1/2020 12:00:00 AM,P200010092,4,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,01:03:05,415,DISTURBANCE,CAN,Canceled,NaN,[0]-[100] PALM VALLEY BL,San Jose,CA,2020-01-01,01:03:05,01/01,2020
2,20200101021613PS,7981725,1/1/2020 12:00:00 AM,P200010134,5,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,01:50:32,647F,DRUNK IN PUBLIC,N,No report required; dispatch record only,NaN,S 1ST ST & E SAN SALVADOR ST,San Jose,CA,2020-01-01,01:50:32,01/01,2020
3,20200101021628PS,7981722,1/1/2020 12:00:00 AM,P200010131,2,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,01:49:04,415A,"DISTURBANCE, FIGHT",N,No report required; dispatch record only,NaN,NERDY AV & CAS DR,San Jose,CA,2020-01-01,01:49:04,01/01,2020
4,20200101021642PS,7981640,1/1/2020 12:00:00 AM,P200010064,2,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,00:43:04,911UNK,UNK TYPE 911 CALL,N,No report required; dispatch record only,NaN,[0]-[100] MERIDIAN AV,San Jose,CA,2020-01-01,00:43:04,01/01,2020
